In [1]:
import pandas as pd
import numpy as np
import pickle
from IPython.display import display
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Finding Repeated Listings

This notebook contains the code for duplicate detection and removal. Pairwise similarity of description texts are calculated for each postal code region. Then, the similar listings are clustered by the multiplication of number of rooms and living area. The number of clusters which maximize silouhette score are considered to be optimal. A unique index is assigned to each cluster of similar listing descriptions. The unique indices represent repeated listings. The weeks on the market of the repeated listings are not aggregated, only the first occurrence is chosen instead. See 1_DataCleaning.ipynb for details on data cleaning and removal of repeated listings. For further information, please refer to Appendix A.







In [2]:
all_cities_buy = pd.read_pickle("data_final/all_cities_buy.pkl")

In [3]:
all_cities_buy.shape

(1555425, 88)

In [9]:
all_cities_buy.head()

,dw_created_at,dw_modified_at,dw_is_active,source_record_id,source_type,source_name,address_geocoded_accuracy,address_geocoded_city,address_geocoded_country,address_geocoded_full,address_geocoded_geometry,address_geocoded_housenr,address_geocoded_postalcode,address_geocoded_street,address_lat,address_lng,apartment_floor_detail,apartment_floor_type,apartmentbuilding_residentials_count,balcony_available,balcony_count_total,balcony_south,basement_available,bath_daylight,bath_quality,construction_accessibility,construction_elevator,construction_quality_object,construction_stage,construction_yearfinished,energy_heating_fireplace,energy_heating_floor,energy_heating_type,energy_ownsolar,energy_type_list,interior_flooring,interior_furnished,interior_quality,investment_rent_active,investment_rent_available,kitchen_available,object_category,object_type,outsideinstallations_conservatory_available,outsideinstallations_garden_available,outsideinstallations_terrace_available,parking_available,parking_total,plot_area_total,plot_lease_available,plot_lease_eternal,state_firstoccupancy,state_modernisation_required,state_object,state_preservation,state_restoration_firstoccupancy,state_restoration_required,state_restoration_structural,structure_area_living,structure_floors,structure_height_building,structure_rooms_baths,structure_rooms_guesttoilets,structure_rooms_living,expose_price_sqm,expose_price_total,expose_offline_since,expose_online_since,expose_online_until,expose_type,expose_urls,expose_content_images,meta_source_id,meta_source_type,address_original_postalcode,energy_certificate_type,energy_certificate_value,state_restoration_year,expose_broker_company,expose_broker_name,expose_content_text,offline_corrected,online_corrected,weeks_on_market,month_advertised,year_advertised,expose_content_text_tokens,token_impute
0,2019-09-10 12:08:38.849021,2019-09-10 12:08:38.849021,True,SrYWXg06tFIw0RYaM59YHrorK0Q=,1,historic,DISTRICT_LARGE,München,None,None,None,None,81829,None,48.133853,11.688066,NaN,None,NaN,True,None,None,True,False,None,None,False,None,ACTIVE_CONSTRUCTION,NaN,None,True,OTHER,None,[HEAT_PUMP],None,False,None,False,None,False,HOUSE,ONE_TWO_FAMILY_HOUSE,None,True,False,True,NaN,287.0,None,None,False,None,None,False,None,False,None,140.00,4.0,None,NaN,1,6.0,6900.000000,966000.0,2018-03-27 22:00:00,2018-01-18 23:00:00,2018-03-26 22:00:00,BUY,None,None,None,None,None,NONE,NaN,NaN,Stadthaus München Wohnbau & Immobilien GmbH,None,Objekt\n======\n\nAuf einem sehr schönen und g...,2018-03-26,2018-01-15,11.0,1,2018,objekt schon gut geleg 778 gross grundstuck im...,NaN
1,2019-09-10 12:08:38.849021,2019-09-10 12:08:38.849021,True,TZW7jHPBUupJ6NJ0WKtFJkdqy50=,1,historic,POSTAL_CODE,Bad Berka,None,None,None,None,99438,None,50.888310,11.254873,NaN,None,NaN,True,None,None,False,False,None,None,False,None,ACTIVE_CONSTRUCTION,2019.0,None,False,OTHER,None,[ELECTRICITY],None,False,None,False,None,False,HOUSE,None,None,False,False,True,NaN,NaN,None,None,False,None,None,False,None,False,None,249.00,NaN,None,1.0,0,6.0,3015.030120,NaN,2018-04-20 22:00:00,2018-04-14 22:00:00,2018-04-19 22:00:00,BUY,None,None,None,None,None,NONE,14.7,NaN,"Verkaufsniederlassung Erfurt, KH Massivhaus Dr...",None,Objektbeschreibung\nDas Bauhaus Ixeo von Kern-...,2018-04-16,2018-04-09,2.0,4,2018,objektbeschreib bauhaus ixeo uberzeugt stilvol...,NaN
2,2019-09-10 12:08:38.849021,2019-09-10 12:08:38.849021,True,TyEV5wOVoBktRskajCBtaFtirpM=,1,historic,POSTAL_CODE,Dinslaken,None,None,None,None,46539,None,51.575197,6.797602,1.0,UPPER_FLOOR,NaN,True,None,None,True,False,None,None,False,None,FINISHED,1973.0,None,False,CENTRAL,None,[DISTRICT_BASED],None,False,None,False,None,False,APARTMENT,REGULAR_APARTMENT,None,False,False,False,NaN,NaN,None,None,False,None,None,False,None,None,None,86.46,NaN,None,NaN,1,3.5,1781.170483,154000.0,NaT,2018-04-10 22:00:00,2018-05-16 22:00:00,BUY,None,[https://valuation-scraper.s3.amazonaws.com/38...,None,None,None,REQUIREMENT_BASED,259.2,NaN,LBS Immobili

In [16]:
all_cities_buy.online_corrected[all_cities_buy.online_corrected < "2019-03-01"].shape

(1072693,)

In [12]:
all_cities_buy[(all_cities_buy.online_corrected >= "2018-01-01") & (all_cities_buy.online_corrected < "2019-01-01")].shape

(940847, 88)

In [5]:
all_cities_buy.structure_rooms_living.isna().sum()

60373

In [6]:
all_cities_buy = all_cities_buy[~all_cities_buy.address_geocoded_postalcode.isna()]

In [7]:
all_cities_buy = all_cities_buy[~all_cities_buy.structure_rooms_living.isna()]

In [8]:
all_cities_buy = all_cities_buy[~all_cities_buy.structure_area_living.isna()]

In [9]:
all_cities_buy.shape

(1494464, 88)

In [10]:
all_cities_buy["obj_unique_id"] = "unknown"
all_cities_buy["obj_unique_id_clus"] = "unknown"





In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer


def pairwise_similarity(text):
    tfidf = TfidfVectorizer().fit_transform(text)
    pairwise_similarity = tfidf * tfidf.T
    return(pairwise_similarity.toarray())







In [12]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


def clusterer(x):

    x = np.array(x).reshape(-1, 1)
    scores = []
    if x.shape[0]<100:
        r = x.shape[0]
    else:
        r = 100
    for n in range(1,r+1):
        cl = KMeans(n_clusters=n).fit(x)
        preds = cl.predict(x)
        try:
            score=silhouette_score(x,preds, metric='euclidean')
        except:
            score=1000.0
        scores.append(score)
    scores = np.array(scores)
    if all(scores == 1000.0):
        n_clu = 1
    elif 1000.0 in scores:
        idx_clu = np.where(scores == np.max(scores[scores != 1000.0]))[0]
        x_length = np.array(list(range(1,r+1)))
        n_clu = x_length[idx_clu][0]
    else:
        idx_clu = np.where(scores == np.max(scores))[0]
        x_length = np.array(list(range(1,r+1)))
        n_clu = x_length[idx_clu][0]
        
    final_cl = KMeans(n_clusters=n_clu).fit(x)
    final_preds = final_cl.predict(x)
    return(final_preds)




In [13]:


def duplicate_finder(x):
    temp = x.copy()
    mat = pairwise_similarity(temp["expose_content_text_tokens"])
    assigned_rows = np.array(list())
    for i in range(temp.shape[0]):
        row = np.where(mat[i]> 0.9)[0]
        cond = pd.Series(row).isin(assigned_rows).values
        row = row[~cond]
        if row.shape[0] == 0:
            continue
        new_id = temp.iloc[row[0],:]["source_record_id"]
        temp.iloc[row,temp.columns.get_loc("obj_unique_id")] = new_id
        assigned_rows = np.append(assigned_rows,row)
        
        # Clustering
        if row.shape[0] == 1:
            temp.iloc[row,temp.columns.get_loc("obj_unique_id_clus")] = temp.iloc[row,temp.columns.get_loc("obj_unique_id")]
            continue
        
        living_area = temp.iloc[row,temp.columns.get_loc("structure_area_living")]
        rooms = temp.iloc[row,temp.columns.get_loc("structure_rooms_living")] 
            
        mult = living_area * rooms
        if all(mult == np.mean(mult)):
            temp.iloc[row,temp.columns.get_loc("obj_unique_id_clus")] = temp.iloc[row,temp.columns.get_loc("obj_unique_id")]
            continue
        
        clus_id = temp.iloc[row,temp.columns.get_loc("obj_unique_id")].astype("str") +"clu" +clusterer(mult).astype(str)
        temp.iloc[row,temp.columns.get_loc("obj_unique_id_clus")] = clus_id
        
    return temp





In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()


all_cities_buy_new = all_cities_buy.groupby("address_geocoded_postalcode").progress_apply(duplicate_finder)






In [ ]:
all_cities_buy_new = all_cities_buy_new.reset_index(level=[1],drop=True)

In [5]:
all_cities_buy_new = all_cities_buy_new.reset_index(drop=True)

In [14]:
#all_cities_buy_new.to_pickle("data_final/all_cities_buy_new.pkl")

In [2]:
all_cities_buy_new = pd.read_pickle("data_final/all_cities_buy_new.pkl")

In [12]:
all_cities_buy_new[all_cities_buy_new.source_record_id != all_cities_buy_new.obj_unique_id]

,dw_created_at,dw_modified_at,dw_is_active,source_record_id,source_type,source_name,address_geocoded_accuracy,address_geocoded_city,address_geocoded_country,address_geocoded_full,address_geocoded_geometry,address_geocoded_housenr,address_geocoded_postalcode,address_geocoded_street,address_lat,address_lng,apartment_floor_detail,apartment_floor_type,apartmentbuilding_residentials_count,balcony_available,balcony_count_total,balcony_south,basement_available,bath_daylight,bath_quality,construction_accessibility,construction_elevator,construction_quality_object,construction_stage,construction_yearfinished,energy_heating_fireplace,energy_heating_floor,energy_heating_type,energy_ownsolar,energy_type_list,interior_flooring,interior_furnished,interior_quality,investment_rent_active,investment_rent_available,kitchen_available,object_category,object_type,outsideinstallations_conservatory_available,outsideinstallations_garden_available,outsideinstallations_terrace_available,parking_available,parking_total,plot_area_total,plot_lease_available,plot_lease_eternal,state_firstoccupancy,state_modernisation_required,state_object,state_preservation,state_restoration_firstoccupancy,state_restoration_required,state_restoration_structural,structure_area_living,structure_floors,structure_height_building,structure_rooms_baths,structure_rooms_guesttoilets,structure_rooms_living,expose_price_sqm,expose_price_total,expose_offline_since,expose_online_since,expose_online_until,expose_type,expose_urls,expose_content_images,meta_source_id,meta_source_type,address_original_postalcode,energy_certificate_type,energy_certificate_value,state_restoration_year,expose_broker_company,expose_broker_name,expose_content_text,offline_corrected,online_corrected,weeks_on_market,month_advertised,year_advertised,expose_content_text_tokens,token_impute,obj_unique_id,obj_unique_id_clus
3,2019-08-28 21:28:36.469330,2019-09-19 14:49:02.233626,True,1789908,2,ImmobilienScout24.de,None,Leisnig,Germany,"Chemnitzer Str. (00102) 29, 04703 Leisnig, Ger...",0101000020E6100000978BF84ECCDA2940CEDF84420494...,00102),00102,Chemnitzer Str. (,51.156380,12.927340,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,HOUSE,MULTI_FAMILY_HOUSE,None,None,None,False,NaN,300.0,False,False,None,None,None,None,None,None,None,200.00,NaN,None,NaN,0,8.0,190.000000,38000.0,2018-06-25 00:00:00,2018-06-28 00:00:00,2018-06-25 00:00:00,BUY,https://www.immobilienscout24.de/105770537,[https://d13xqcsmyavurl.cloudfront.net/immobil...,None,None,None,NONE,NaN,NaN,Real Estate Virtual Solutions UG (haftungsbesc...,Ot,"Energieausweis: BAUDENKMAL, ENERGIEAUSWEIS NIC...",2018-06-25,2018-06-25,1.0,6,2018,energieausweis baudenkmal energieausweis vorge...,NaN,2074824,2074824
10,2019-08-28 21:28:36.469330,2019-09-19 14:49:02.233626,True,3633260,2,ImmobilienScout24.de,None,Dresden,Germany,"Seminarstraße 15, 01067 Dresden, Germany",0101000020E6100000E2E47E87A2702B40A5315A475587...,15,01067,Seminarstraße,51.057290,13.719990,3.0,None,NaN,True,None,None,None,None,None,None,True,None,FINISHED_RECENTLY,2019.0,None,False,OTHER,None,[DISTRICT_BASED],None,None,None,False,FREE_NEW,None,APARTMENT,REGULAR_APARTMENT,None,None,None,True,NaN,NaN,False,False,True,FEW,FINISHED_RECENTLY,None,False,False,None,43.00,NaN,None,1.0,0,2.0,4511.629883,194000.0,2019-01-22 00:00:00,2018-12-28 00:00:00,2019-01-22 00:00:00,BUY,https://www.immobilienscout24.de/109071883,[https://d13xqcsmyavurl.cloudfront.net/immobil...,None,None,None,REQUIREMENT_BASED,44.0,NaN,Immolounge GenoScout GmbH,Tanja Gallinger,"Absolute Stadtnähe, alles in der Umgebung fußl...",2019-01-28,2018-12-24,6.0,12,2018,absolut stadtnah umgeb fusslauf erreichbar idy...,absolut stadtnah umgeb fusslauf erreichbar idy...,4361933,4361933
24,2019-08-28 21:28:36.469330,2019-09-19 14:49:02.233626,True,2136886,2,ImmobilienScout24.de,None,Dresden,Germany,"Seminarstraße 28, 01067 Dresden, Germany",0101000020E6100000E2E47E87A2702B40

# Some Examples

In [6]:
all_cities_buy_new[all_cities_buy_new.obj_unique_id == "1617753"][["obj_unique_id_clus","structure_area_living",
                                                                   "structure_rooms_living",
                                                                   "expose_price_total","expose_content_text_tokens",
                                                                   "obj_unique_id",
                                                                  "offline_corrected",
                                                                   "online_corrected",
                                                                   "weeks_on_market"]]

,obj_unique_id_clus,structure_area_living,structure_rooms_living,expose_price_total,expose_content_text_tokens,obj_unique_id,offline_corrected,online_corrected,weeks_on_market
1489900,1617753clu3,52.0,3.0,36000.0,wohnung nr 63 im 1 og recht 52 besteh 3 zimm k...,1617753,2018-06-11,2018-06-11,1.0
1489906,1617753clu9,40.0,2.0,26000.0,wohnung nr 64 im 1 og link 40 besteh 2 zimm ku...,1617753,2018-06-11,2018-06-11,1.0
1489921,1617753clu9,40.0,2.0,26000.0,wohnung nr 64 im 1 og link 40 besteh 2 zimm ku...,1617753,2018-06-18,2018-06-18,1.0
1489931,1617753clu3,52.0,3.0,36000.0,wohnung nr 63 im 1 og recht 52 besteh 3 zimm k...,1617753,2018-06-18,2018-06-18,1.0
1489949,1617753clu2,38.0,2.0,24000.0,wohnung nr 62 im eg link 38 besteh 2 zimm kuch...,1617753,2018-06-11,2018-06-11,1.0
1489952,1617753clu7,40.0,1.0,26000.0,wohnung nr 57 im 1 og recht 40 besteh zimm kuc...,1617753,2018-06-18,2018-06-18,1.0
1489980,1617753clu4,28.0,1.0,22000.0,wohnung nr 66 im 2 og link 28 besteh zimm kuch...,1617753,2018-06-18,2018-06-18,1.0
1489985,1617753clu0,68.0,3.0,39000.0,wohnung nr 1 im eg recht besteh 3 zimm flur ku...,1617753,2019-01-14,2019-01-14,1.0
1490013,1617753clu0,68.0,3.0,39000.0,wohnung nr 1 im eg recht besteh 3 zimm flur ku...,1617753,2019-01-07,2019-01-07,1.0
1490019,1617753clu5,50.0,2.0,23000.0,wohnung nr 61 im eg recht 50 besteh 2 zimm kuc...,1617753,2018-06-11,2018-06-11,1.0


In [7]:
all_cities_buy_new[all_cities_buy_new.obj_unique_id == "6651815"][["obj_unique_id_clus","structure_area_living",
                                                                   "structure_rooms_living",
                                                                   "expose_price_total","expose_content_text_tokens",
                                                                   "obj_unique_id",
                                                                  "offline_corrected",
                                                                   "online_corrected",
                                                                   "weeks_on_market"]]

,obj_unique_id_clus,structure_area_living,structure_rooms_living,expose_price_total,expose_content_text_tokens,obj_unique_id,offline_corrected,online_corrected,weeks_on_market
1493491,6651815clu1,47.0,2.0,23400.0,wohnung nr 5 im 1 og mitt 47 besteh 2 zimm kuc...,6651815,2019-06-17,2019-06-17,1.0
1493495,6651815clu1,47.0,2.0,23300.0,wohnung nr 1 im eg link 47 besteh 2 zimm kuch ...,6651815,2019-05-13,2019-05-13,1.0
1493500,6651815clu0,63.0,3.0,31500.0,wohnung nr 3 im eg recht 63 besteh 3 zimm kuch...,6651815,2019-06-10,2019-06-03,2.0
1493502,6651815clu0,63.0,3.0,31500.0,wohnung nr 6 im 1 og recht 63 besteh 3 zimm ku...,6651815,2019-04-15,2019-04-15,1.0
1493504,6651815clu1,47.0,2.0,23400.0,wohnung nr 2 im eg mitt 47 wfl besteh 2 zimm k...,6651815,2019-05-06,2019-05-06,1.0
1493505,6651815clu1,47.0,2.0,23300.0,wohnung nr 1 im eg link 47 besteh 2 zimm kuch ...,6651815,2019-05-06,2019-05-06,1.0
1493506,6651815clu0,63.0,3.0,31500.0,wohnung nr 6 im 1 og recht 63 besteh 3 zimm ku...,6651815,2018-10-29,2018-10-29,1.0
1493509,6651815clu0,63.0,3.0,31500.0,wohnung nr 3 im eg recht 63 besteh 3 zimm kuch...,6651815,2018-10-29,2018-10-29,1.0
1493516,6651815clu1,47.0,2.0,23400.0,wohnung nr 2 im eg mitt 47 wfl besteh 2 zimm k...,6651815,2019-04-15,2019-04-15,1.0
1493518,6651815clu1,47.0,2.0,23400.0,wohnung nr 2 im eg mitt 47 wfl besteh 2 zimm k...,6651815,2019-04-29,2019-04-29,1.0


In [13]:
all_cities_buy_new[all_cities_buy_new.obj_unique_id == "5708217"][["obj_unique_id_clus","structure_area_living",
                                                                   "structure_rooms_living",
                                                                   "expose_price_total","expose_content_text_tokens",
                                                                   "obj_unique_id",
                                                                  "offline_corrected",
                                                                   "online_corrected",
                                                                   "weeks_on_market"]]

,obj_unique_id_clus,structure_area_living,structure_rooms_living,expose_price_total,expose_content_text_tokens,obj_unique_id,offline_corrected,online_corrected,weeks_on_market
1494254,5708217clu0,49.0,2.0,14000.0,wohnung nr 9 im 1 og link besteh 2 zimm kuch b...,5708217,2019-04-29,2019-04-29,1.0
1494258,5708217clu0,49.0,2.0,14000.0,wohnung nr 9 im 1 og link besteh 2 zimm kuch b...,5708217,2019-05-27,2019-05-27,1.0
1494260,5708217clu0,49.0,2.0,14000.0,wohnung nr 9 im 1 og link besteh 2 zimm kuch b...,5708217,2019-08-05,2019-07-29,2.0
1494264,5708217clu0,49.0,2.0,14000.0,wohnung nr 9 im 1 og link besteh 2 zimm kuch b...,5708217,2019-06-17,2019-06-17,1.0
1494269,5708217clu1,49.1,2.0,14000.0,wohnung nr 9 im 1 og link besteh 2 zimm kuch b...,5708217,2019-07-01,2019-07-01,1.0
1494271,5708217clu1,49.1,2.0,14000.0,wohnung nr 9 im 1 og link besteh 2 zimm kuch b...,5708217,2019-07-15,2019-07-08,2.0
1494275,5708217clu0,49.0,2.0,14000.0,wohnung nr 9 im 1 og besteh 2 zimm kuch bad so...,5708217,2019-02-25,2019-02-25,1.0
1494282,5708217clu0,49.0,2.0,14000.0,wohnung nr 9 im 1 og besteh 2 zimm kuch bad so...,5708217,2019-03-25,2019-03-25,1.0
1494286,5708217clu0,49.0,2.0,14000.0,wohnung nr 9 im 1 og link besteh 2 zimm kuch b...,5708217,2019-05-06,2019-05-06,1.0
1494288,5708217clu0,49.0,2.0,14000.0,wohnung nr 9 im 1 og link besteh 2 zimm kuch b...,5708217,2019-08-05,2019-08-05,1.0
